### **ECO 541: Problem Set 1**  

* *Original Author:* Emily Merola
* *First created:* 2023.09.25
* *Collaborated with:* Cameron Ricciardi, Calvin Jahnke
* *All other work for this problem set is either handwritten, or completed in Stata*

#### **Part 2, question 6** 

Now suppose you learn from industry sources that in a fraction of the markets in your data
behavior is collusive while in the others behavior is Cournot. Estimate the fraction of markets
(λ) that are collusive (The readings should help once again).

In [1]:
# Housekeeping
using LinearAlgebra, Plots, StatsPlots, SparseArrays, Random, Statistics, Parameters
using BenchmarkTools, PiecewiseLinearApprox, Optim, Distributions, NLSolversBase
using CSV, DataFrames
using GLM

In [2]:
# Bring on the dataset
data = CSV.read("/Users/emilymerola/Documents/git/IO_fa23_psets/ps1_data_2023.csv", DataFrame);
first(data,5) #load 5 rows & inspect

,N,S,W,Q,P
,Int64,Int64,Float64,Float64,Float64
1,2,45,1.32,7.89,20.39
2,2,35,1.34,8.37,21.88
3,2,24,4.7,3.16,25.02
4,2,41,2.34,6.58,33.02
5,2,68,2.33,7.39,36.43


In [3]:
## function to run OLS and get parameter estimates for A/gamma/deltas & calc residuals
# take alpha as given 
# ("suppose we knew what alpha was. then the following only has exogenous vars on the RHS, and OLS = MLE")
# whole point is to simplify the search procedure re: MLE for all of these parameters.

function ols(X::Vector)
    # pull what we need from the original dataframe
    y_demand = data.P + ((X[1]) .* data.Q)      # same as below. setting aside to make regression equations below clear.
    y_collusive = data.P .- ((X[1]) .* data.Q)
    y_cournot = data.P .- ((X[1] ./ data.N) .* data.Q)
    W = data.W
    S = data.S

    df = DataFrame(y0 = y_demand, y1 = y_collusive, y2 = y_cournot, W = W, S = S)

    # demand
    ols0 = lm(@formula(y0~S), df)
    A = coef(ols0)[1,1]
    gamma = coef(ols0)[2,1]

    # supply: collusive case
    ols1 = lm(@formula(y1~W), df)
    δ0_1 = coef(ols1)[1,1]
    δ1_1 = coef(ols1)[2,1]

    # supply: cournot case
    ols2 = lm(@formula(y2~W), df)
    δ0_2 = coef(ols2)[1,1]
    δ1_2 = coef(ols2)[2,1]

    # predictions for y-hats
    y_hat0 = predict(ols0)
    y_hat1 = predict(ols1)
    y_hat2 = predict(ols2)

    # residuals
    eps0 = df.y0 - y_hat0
    eps1 = df.y1 - y_hat1
    eps2 = df.y2 - y_hat2

    eps_supply = (X[2] .* eps1) .+ ((1-X[2]) .* eps2)

    return Δ = (A=A, gamma=gamma, δ0_1=δ0_1, δ1_1=δ1_1, δ0_2=δ0_2, δ1_2=δ1_2, eps0=eps0, eps1=eps1, eps2=eps2, eps_supply)
end

ols (generic function with 1 method)

In [4]:
# Function to generate our optimal parameters guess
function optimal_params(Z::Vector)
    X = [Z[1],Z[2]]

    ols_res = ols(X)

    sig1 = exp(Z[3])
    sig2 = exp(Z[4])

    lk_1 = sum(logpdf.(Normal(0,sig1), ols_res.eps0))
    lk_2 = sum(logpdf.(Normal(0,sig2), ols_res.eps_supply))

    lk_1 = -lk_1
    lk_2 = -lk_2

    lklhd = lk_1 + lk_2 
    return Ω = (lklhd = lklhd) 
end

optimal_params (generic function with 1 method)

In [5]:
## Solve for lambda 

## initial values
    # set the seed (not completely necessary)
    Random.seed!(123)

    # first set of w guesses, first guess of lambda
    w0 = rand(Uniform(0.0,1.0), 500)

    # generate first guess of lambda
    λ0 = sum(w0)/500

## Update lambdas until they converge
diff = 1
while diff > 0.001

    # our guess (note that only the lambda needs to update)
    Z = [-4, λ0, 2.302585092994046, 2.4849066497880004]

    # calculate the optimal parameters for the given guess
    par_res = optimize(optimal_params, Z)

    # pull out "optimal parameters" we just found, also use them (alpha) to calculate the implied errors we observe
    opt_sig2 = Optim.minimizer(par_res)[4]
    X2 = [Optim.minimizer(par_res)[1], Optim.minimizer(par_res)[2]]
    ols_res = ols(X2)


    # adjust to next guess of w's using Bayes' rule
    top = λ0 * pdf.(Normal(0,opt_sig2), ols_res.eps1)
    bottom = top + (1-λ0)*pdf.(Normal(0,opt_sig2), ols_res.eps2)
    w1 = top./bottom

    # calculate the new lambda
    λ1 = sum(w1)/500

    # compare to previous lambda
    diff = λ1 - λ0

    # # update for next iteration (if diff is too big)
    λ0 = λ1

end

λ0

0.2847718765335344